#Sheet Collection
######`s_responses` is the Responses Sheet
######`s_ref_match_stats` is the Refined Match States Sheet
######`s_calculations` is the Calculations Sheet
###### `s_team_stats` is the Team Statistics Sheet
###### etc.


In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials


# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
client = gspread.authorize(creds)

# Find a workbook by name and open the first sheet
sheet = client.open("Robotics OPGG")

# Grab Each Worksheet
s_responses = sheet.worksheet("Form Responses 1")
s_ref_match_stats = sheet.worksheet("Refined Match Stats")
s_calculations = sheet.worksheet("Calculations")
s_team_stats = sheet.worksheet("Team Stats")
s_jacob_test_stats = sheet.worksheet("Jacob's Stat Testing")

print("Startup complete!")

In [ ]:
#Installs a stats package for easy use
!pip install scipy


# One-mean t-test
##### Allows for calculating the probability that a data value occurs given our dataset. Ex: Say you want to find the probability that a robot will shoot 30 balls during the next match and you already know that the robot shot 12,15,45,47,23,20, and 34 shots during the last matches. 

#####First get the mean of the data and the standard deviation of the data

#####Then find the standard deviation for the t-model

#####Then find the t-score of the desired value, data mean, and standard deviation of the t-model

#####Finally, calculate the tcdf of the data to get the probability that your desired value occurs





In [ ]:
import math  
from scipy.stats import t
from scipy.stats import norm
from decimal import Decimal

def get_mean(spreadsheet, column):
  values = spreadsheet.col_values(column)
  #print(spreadsheet.cell(1, 1).value)
  #print(values)
  sum = 0
  for val in values:
    value = float(val)
    #print("Value is "+str(val))
    sum += value
  mean = sum/len(values)
  #print(mean)
  return mean

def get_sd(spreadsheet, column):
  values = spreadsheet.col_values(column)
  sum_of_diffs = 0
  mean = get_mean(spreadsheet,column)
  for val in values:
    value = float(val)
    sum_of_diffs += pow(value-mean,2)
  return math.sqrt(sum_of_diffs/(len(values)-1))

def get_tsd(spreadsheet,column):
  values = spreadsheet.col_values(column)
  tsd = get_sd(spreadsheet,column)/math.sqrt(len(values))
  #print(tsd)
  return tsd

def get_tscore(spreadsheet,column,desired_val):
  return (get_mean(spreadsheet,column)-desired_val)/get_tsd(spreadsheet,column)

def get_zscore(spreadsheet,column,desired_val):
  return (get_mean(spreadsheet,column)-desired_val)/get_sd(spreadsheet,column)

def get_normal_probability(spreadsheet, column, desired_val):
  values = spreadsheet.col_values(column)
  zscore = -abs(get_zscore(spreadsheet,column,desired_val))
  return norm.cdf(zscore)*2

#Tells if a value is unusual in a dataset
def is_unusual(spreadsheet, column, desired_val):
  zscore = get_zscore(spreadsheet,column,desired_val)
  if (abs(zscore) >= 2):
    return true
  return false

#Does a two-tailed test with z-score instead

#Returns the probability of a value occuring within our dataset
#Notice: Does a two tail p-value (hence the *2 at the end)
def get_pvalue(spreadsheet, column, desired_val):
  values = spreadsheet.col_values(column)
  tscore = -abs(get_tscore(spreadsheet,column,desired_val))
  #print(tscore)
  return t.cdf(tscore,len(values)-1)*2

#Does a two-tailed one-mean t-test 
#alpha represents the required p-value for the value to be plausibly in our data set
#alpha should be a decimal less than or equal to 1
#typically alpha should be 0.05
def t_test(spreadsheet, column, desired_val, alpha):
  pvalue = get_pvalue(spreadsheet, column, desired_val)
  print("Beginning a two-tailed one-mean t-test with a significance level of α="+str(alpha))
  #When we do a test, we test to see if a hypothesis is true. In this case we test to see if the desired value is the mean
  #We pit it against an alternate hypothesis
  print("Null Hypothesis(H0): μ = "+str(desired_val)+" | The mean of the data is "+str(desired_val))
  print("Alternate Hypothesis(HA): μ ≠ "+str(desired_val)+" | The mean of the data is not "+str(desired_val))
  print()
  if (pvalue < alpha):
    print("With a p-value of "+str(pvalue)+"(less than "+str(alpha)+"), we reject the null hypothesis.")
    print("There is sufficient evidence that the true mean of the dataset is not "+str(desired_val)+".")
    print("In other words, it is unusual if "+str(desired_val)+" is the true mean")
  else:
    print("With a p-value of "+str(pvalue)+"(greater than or equal to "+str(alpha)+"), we can't reject the null hypothesis.")
    print("There is insufficient evidence that the true mean of the dataset is not "+str(desired_val)+".")
    print("In other words, it is not unusual if "+str(desired_val)+" is the true mean")
  print()

#Gets a confidence interval using a t-model
#Confidence should be a decimal less than or equal to 1
#Typically confidence is 0.95
def t_interval(spreadsheet, column, confidence):
  values = spreadsheet.col_values(column)
  mean = get_mean(spreadsheet,column)
  tcrit = t.ppf(confidence+(1-confidence)/2,len(values)-1)
  tsd = get_tsd(spreadsheet,column)
  lower_bound = mean-tcrit*tsd
  upper_bound = mean+tcrit*tsd
  print("Beginning a one-mean t-interval")
  print()
  print("We are "+str(100*confidence)+"% confident that the true mean of the data set is between "+str(lower_bound)+" and "+str(upper_bound)+".")
  

#Checks the probability of a value occuring using a normal model


  
#Test the probability of the value occuring

t_test(s_jacob_test_stats,1,5,0.05)
t_interval(s_jacob_test_stats,1,0.95)




Beginning a two-tailed one-mean t-test with a significance level of α=0.05
Null Hypothesis(H0): μ = 5 | The mean of the data is 5
Alternate Hypothesis(HA): μ ≠ 5 | The mean of the data is not 5

With a p-value of 6.502521116681945e-07(less than 0.05), we reject the null hypothesis.
There is sufficient evidence that the true mean of the dataset is not 5.
In other words, it is unusual if 5 is the true mean

Beginning a one-mean t-interval

We are 95.0% confident that the true mean of the data set is between 18.588567069368906 and 30.729432930631084.


In [ ]:
from scipy import stats
from scipy.stats import norm
import math


#This code section sets data on the spreadsheet
teamsDictionary = dict()
#This stores the row for the team on the teamstats sheet
teamsNumberDictionary = dict()
teamsGamesDictionary = dict()
teamsLinesDictionary = dict()

#Stores stuff for Probability Finding
teamsAutonBallQntyAvgs = dict()
teamsAutonBallQntySD = dict()

#Flexible Dictionaries
teamsVals = dict()
teamsAvgs = dict()
teamsSD = dict()

#Auton data
teamsAutonScores = dict()

#Teleop data
teamTeleopScores = dict()
#Gathers data from all the teams

#check in dict, if the teamNumber value matches the parameter, print out that row?
#Works: TRUE
def init_teamsMatrix():
  values = s_ref_match_stats.get_all_values()
  for index in range(1,len(values)):
    row = values[index]
    teamNumber = int(row[2])
    if teamNumber in teamsDictionary:
      teamsDictionary[teamNumber].append(row)
    else:
      teamsDictionary[teamNumber] = [row]
  print(teamsDictionary)

#Goes through the teams dictionary and puts ONLY the team number on the team stats sheet
#Works: TRUE
def init_Team_Stats_Sheet():
  keys = list(teamsDictionary)
  keys.sort()

  #grabs the values from the second row to the end of the teams
  loc_stats_sheet = s_team_stats.range('A2:A' + str(len(keys) + 1))

  for index in range(2,len(keys)+2):
    teamNumber = keys[index-2]
    #loc_stats_sheet.update_cell(index,1,teamNumber)
    loc_stats_sheet[index-2].value = teamNumber;
    teamsNumberDictionary[teamNumber] = index-2

  s_team_stats.update_cells(loc_stats_sheet)

#Puts the lines crossed for each team in the sheet
#Also updates the number of games each player plays
#Works: TRUE
def getLinesCrossed():

  keys = list(teamsDictionary)
  keys.sort()

  loc_games = s_team_stats.range('B2:B' + str(len(keys) + 1))
  loc_lines = s_team_stats.range('C2:C' + str(len(keys) + 1))

  for teamNumber in teamsDictionary:
    count = 0
    gameCount = 0
    #print("Printing stats of team #"+str(teamNumber))
    for game in teamsDictionary.get(teamNumber):
      gameCount+=1
      if (game[5]=='1'):count+=1
    #After getting the total number of lines crossed by this team, we need to add it to the sheet
    loc_lines[teamsNumberDictionary[teamNumber]].value = (round(count/float(gameCount)*100,3))
    loc_games[teamsNumberDictionary[teamNumber]].value = gameCount 
    teamsLinesDictionary[teamNumber] = (count/float(gameCount))
    teamsGamesDictionary[teamNumber] = gameCount
  s_team_stats.update_cells(loc_lines)
  s_team_stats.update_cells(loc_games)

def getAutonScoring():
  loc_lower = s_team_stats.range('D2:D' + str(len(teamsDictionary) + 1))
  loc_upper = s_team_stats.range('E2:E' + str(len(teamsDictionary) + 1))
  loc_inner = s_team_stats.range('F2:F' + str(len(teamsDictionary) + 1))
  loc_auton = s_team_stats.range('G2:G' + str(len(teamsDictionary) + 1))
  for teamNumber in teamsDictionary:
    lower = 0
    upper = 0
    inner = 0
    gameCount = teamsGamesDictionary[teamNumber]
    for game in teamsDictionary.get(teamNumber):
      lower += int(game[6]) 
      upper += int(game[7])
      inner += int(game[8])
    #Column 4 is the average auton lower ball score
    loc_lower[teamsNumberDictionary[teamNumber]].value = lower/float(gameCount)
    #Column 5 is the average auton upper ball score
    loc_upper[teamsNumberDictionary[teamNumber]].value = upper/float(gameCount)
    #Column 6 is the average auton inner ball score
    loc_inner[teamsNumberDictionary[teamNumber]].value = inner/float(gameCount)
    auton_score = (lower*2+upper*4+inner*6)/float(gameCount)+5*teamsLinesDictionary[teamNumber]
    #Column 7 is the average auton score ball score
    loc_auton[teamsNumberDictionary[teamNumber]].value = auton_score
    teamsAutonScores[teamNumber] = auton_score
  s_team_stats.update_cells(loc_lower)
  s_team_stats.update_cells(loc_upper)
  s_team_stats.update_cells(loc_inner)
  s_team_stats.update_cells(loc_auton)

def getTeleopScoring():
  teamCount1 = str(len(teamsDictionary)+1)
  loc_lower = s_team_stats.range('H2:H' + teamCount1)
  loc_upper = s_team_stats.range('I2:I' + teamCount1)
  loc_inner = s_team_stats.range('J2:J' + teamCount1)
  loc_ball_score = s_team_stats.range('K2:K' + teamCount1)
  loc_ball_rate = s_team_stats.range('L2:L' + teamCount1)
  loc_teleop_score = s_team_stats.range('M2:M' + teamCount1)
  loc_teleop_rate = s_team_stats.range('N2:N' + teamCount1)
  loc_rotation = s_team_stats.range('R2:R' + teamCount1)
  loc_position = s_team_stats.range('S2:S' + teamCount1)
  loc_pass_rate = s_team_stats.range('O2:O' + teamCount1)
  loc_defense_ratio = s_team_stats.range('P2:P' + teamCount1)
  loc_hung = s_team_stats.range('T2:T' + teamCount1)
  loc_balanced = s_team_stats.range('U2:U' + teamCount1)
  loc_bal_percent = s_team_stats.range('V2:V' + teamCount1)
  loc_latest_qual = s_team_stats.range('AA2:AA' + teamCount1)
  for teamNumber in teamsDictionary:
    passes = 0
    defenseCount = 0
    lower = 0
    upper = 0
    inner = 0
    rotation = 0
    position = 0
    hung = 0 #25 per robot
    balanced = 0 #whether or not robot hangs
    balancedHung = 0 #15 pts if 2 or more robots hung
    gameCount = teamsGamesDictionary[teamNumber]
    highestQual = 0
    for game in teamsDictionary.get(teamNumber):
      passes += int(game[16])
      lower += int(game[9]) 
      upper += int(game[10])
      inner += int(game[11])
      defenseCount += int(game[22])
      if (game[12] == '1'):
        rotation += 1
      if (game[14] == '1'):
        position += 1
      currentHung = int(game[17])
      hung += currentHung
      if(int(highestQual) < int(game[3])):
        highestQual = game[3]
      if (game[18] == '1'):
        balanced += 1
        if (currentHung >= 2):
          balancedHung += 1
    row = teamsNumberDictionary[teamNumber]
    #Column 8 is the average teleop lower ball score 
    loc_lower[row].value = lower/float(gameCount)
    #Column 9 is the average teleop upper ball score 
    loc_upper[row].value = upper/float(gameCount)
    #Column 10 is the average teleop inner ball score 
    loc_inner[row].value = inner/float(gameCount)
    ball_score = (lower*2+upper*4+inner*6)/float(gameCount)

    #Column 11 is the average teleop ball score
    loc_ball_score[row].value = ball_score
    #Column 12 is the teleop ball scoring rate(teleop ball score/total time)
    loc_ball_rate[row].value = ball_score/135

    teleop_score = (lower*2+upper*4+inner*6+rotation*10+position*20+hung*25+balancedHung*15)/gameCount
    #Column 8 is the average teleop score 
    loc_teleop_score[row].value = teleop_score
    #Column 9 is the average scoring rate for teleop (teleopscore/total time)
    loc_teleop_rate[row].value = teleop_score/135
    #Column 15 is defense ratio
    loc_defense_ratio[row].value = (defenseCount/float(gameCount))*100
    #Column 16 is pass rate
    loc_pass_rate[row].value = str(round((passes/float(gameCount)/135), 3)) + '/sec'
    #Column 18 is rotation control
    loc_rotation[row].value = rotation/float(gameCount)
    #Column 19 is position control
    loc_position[row].value = position/float(gameCount)
    #Column 20 is hung control
    loc_bal_percent[row].value = str((balanced/float(gameCount)) * 100) + '%'
    loc_hung[row].value = hung/float(gameCount)
    #Column 21 is rotation control
    loc_balanced[row].value = str((balancedHung/float(gameCount))*100) + '%'
    loc_latest_qual[row].value = highestQual

  s_team_stats.update_cells(loc_lower)
  s_team_stats.update_cells(loc_upper)
  s_team_stats.update_cells(loc_inner)
  s_team_stats.update_cells(loc_ball_score)
  s_team_stats.update_cells(loc_ball_rate)
  s_team_stats.update_cells(loc_teleop_score)
  s_team_stats.update_cells(loc_teleop_rate)
  s_team_stats.update_cells(loc_rotation)
  s_team_stats.update_cells(loc_position)
  s_team_stats.update_cells(loc_defense_ratio)
  s_team_stats.update_cells(loc_hung)
  s_team_stats.update_cells(loc_bal_percent)
  s_team_stats.update_cells(loc_balanced)
  s_team_stats.update_cells(loc_pass_rate)
  s_team_stats.update_cells(loc_latest_qual)

def getWinRate():
  teamCount1 = str(len(teamsDictionary)+1)
  loc_win = s_team_stats.range('X2:X' + teamCount1)
  loc_rp = s_team_stats.range('Y2:Y' + teamCount1)
  loc_rpex_win = s_team_stats.range('Z2:Z' + teamCount1)
  for teamNumber in teamsDictionary:
    wincount = 0.0
    rp = 0
    rpex = 0
    gameCount = teamsGamesDictionary[teamNumber]
    for game in teamsDictionary.get(teamNumber):
      #check if win
      if int(game[20]) > int(game[23]):
        wincount+=1
      elif int(game[20]) == int(game[23]):
        wincount+=.5
      rp += int(game[25])
      rpex += int(game[24])
    row = teamsNumberDictionary[teamNumber]
    loc_win[row].value = wincount/gameCount
    loc_rp[row].value = rpex/float(gameCount)
    loc_rpex_win[row].value = rp/float(gameCount)
  s_team_stats.update_cells(loc_win)
  s_team_stats.update_cells(loc_rp)
  s_team_stats.update_cells(loc_rpex_win)
      

init_teamsMatrix()
init_Team_Stats_Sheet()
getLinesCrossed()
getAutonScoring()
getTeleopScoring()
getWinRate()


#Flexible standard deviation
#data_type is the type of data (used to store in dictionaries), matrix_cols are the cols that need to be considered, update_col is where we update the standard deviation in the sheet

def get_standard_deviation(data_type, matrix_cols, update_col):
  teamCount = str(len(teamsDictionary)+1)
  loc_update_col = s_team_stats.range(update_col+'2:'+update_col+teamCount)
  for teamNumber in teamsDictionary:
    valueSum = 0
    gameCount = teamsGamesDictionary[teamNumber]
    value_list = []
    for game in teamsDictionary.get(teamNumber):
      for col in matrix_cols:
        value = int(game[col])
        value_list.append(int(game[col]))
        valueSum += int(value)
    teamsVals[data_type+":"+str(teamNumber)] = value_list
    row = teamsNumberDictionary[teamNumber]
    average = valueSum/float(gameCount)
    #print("Mean: "+str(average))
    teamsAvgs[data_type+":"+str(teamNumber)] = average
    averageDiffs = []
    for game in teamsDictionary.get(teamNumber):
      temp_sum = 0
      for col in matrix_cols:
        temp_sum += int(game[col])
      averageDiffs.append(math.pow(temp_sum-average,2))
    sumAverageDiffs = 0
    for averageDiff in averageDiffs:
      sumAverageDiffs += averageDiff
    standardDeviation = math.sqrt(sumAverageDiffs/gameCount)
    teamsSD[data_type+":"+str(teamNumber)] = standardDeviation 
    #print("SD: "+str(standardDeviation))
    loc_update_col[row].value = round(standardDeviation, 3)
  s_team_stats.update_cells(loc_update_col)

#Flexible probability method
#test_values are the values to test against
#tail_types stores the ranges of the values to test. 0 indicates left tail (less than test value), 1 indicates right tail value (greater than test value), and 2 is a left and right tail test(essentially double left tail)
#update_cols are the respective column that stores that test value probability
def get_probability(data_type, nearly_normal_col, test_values, tail_types,update_cols):
  teamCount = str(len(teamsDictionary)+1)
  loc_nearly_normal_col = s_team_stats.range(nearly_normal_col+'2:'+nearly_normal_col+teamCount)
  update_cols_list = []
  index = 0
  for update_col in update_cols:
    update_cols_list.append(s_team_stats.range(update_col+'2:'+update_col+teamCount))
    index += 1
  for teamNumber in teamsDictionary:
    vals = teamsVals[data_type+":"+str(teamNumber)]
    gameCount = teamsGamesDictionary[teamNumber]
    row = teamsNumberDictionary[teamNumber]
    teamAvg = teamsAvgs[data_type+":"+str(teamNumber)]
    teamSD = teamsSD[data_type+":"+str(teamNumber)]
    print("Doing team: "+str(teamNumber))
    #print(update_col+'2:'+update_col+teamCount)
    if gameCount >= 5:
      if teamAvg == 0:
        loc_nearly_normal_col[row].value = "No point in test - mean is 0"
        index = 0
        for update_col in update_cols_list:
          update_col[row].value = 'N/A'
          index += 1
        continue
      elif teamSD == 0:
        loc_nearly_normal_col[row].value = "No point in test - standard deviation is 0"
        index = 0
        for update_col in update_cols_list:
          update_col[row].value = 'N/A'
          index += 1
        continue
      #print("Testing p for team #"+str(teamNumber)+" for datatype: "+data_type)
      #print(vals)
      p=stats.normaltest(vals).pvalue
      #print(p)
      if p<=0.01:
        loc_nearly_normal_col[row].value = 'No (p = ' + str(round(p*100,3)) + '%)'
        index = 0
        for update_col in update_cols_list:
          update_col[row].value = 'N/A'
          index += 1
        continue
      elif p<=0.05:
        loc_nearly_normal_col[row].value = 'Proceed with caution (p = ' + str(round(p*100,3)) + '%)'
      else: 
    
        loc_nearly_normal_col[row].value = 'Yes (p = ' + str(round(p*100,3)) + '%)'
    else:
      loc_nearly_normal_col[row].value = 'Not enough Data'
      print("Not enough Data!")
      index = 0
      for update_col in update_cols_list:
        update_col[row].value = 'N/A'
        index += 1
      continue
    print("Finding probabilities")
    for i in range(0,len(test_values)):
      if (teamSD == 0 ):
        update_cols_list[i][row].value = "Can't Calculate Probability, Standard Deviation is 0"
        continue
      zscore = (test_values[i]-teamAvg)/teamSD
      probability = 0
      #Test for probability of any value below test_value
      if tail_types[i] == 0:
        probability = norm.cdf(zscore)
      #Test for probability of any value above test_value
      elif tail_types[i] == 1:
        probability = 1-norm.cdf(zscore)
      #Test for probability of two tail test (2*probability of any value below test_value)
      else:
        probability = 2*norm.cdf(-math.abs(zscore))
      update_cols_list[i][row].value = str(round(probability*100,3)) + '%'
      print(probability)
  s_team_stats.update_cells(loc_nearly_normal_col)
  index = 0
  for update_col in update_cols_list:
    s_team_stats.update_cells(update_col)




#get_standard_deviation_autonballs()
#get_probability_autonballs()
get_standard_deviation("AutonBalls",[6,7,8],'AD')
get_probability("AutonBalls",'AC',[3],[1],['AE'])
get_standard_deviation("TeleOpBalls",[9,10,11],'AH')
get_probability("TeleOpBalls",'AG',[5,9,29,49],[1,1,1,1],['AI','AJ','AK','AL'])

{8150: [['3/7/2020 11:25:22', '', '8150', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '68', '3', '0', '34', '0', '0'], ['3/7/2020 16:00:27', '', '8150', '22', '24', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '56', '29', '0', '65', '0', '0'], ['3/7/2020 16:01:53', '', '8150', '22', '24', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '56', '29', '0', '65', '0', '0'], ['3/7/2020 17:06:44', '', '8150', '32', '29', '1', '0', '0', '0', '0', '5', '1', '0', '0', '0', '0', '0', '0', '0', '0', '78', '28', '0', '36', '2', '0'], ['', '', '8150', '51', '25', '0', '0', '0', '0', '3', '4', '0', '0', '0', '0', '0', '0', '0', '0', '0', '69', '23', '0', '49', '2', '0'], ['', '', '8150', '51', '25', '0', '0', '0', '0', '3', '4', '0', '0', '0', '0', '0', '0', '0', '0', '0', '69', '23', '0', '49', '2', '2']], 6111: [['3/7/2020 11:20:16', '', '6111', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0

/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  "anyway, n=%i" % int(n))
/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  "anyway, n=%i" % int(n))


Doing team: 8150
Finding probabilities
0.30941169662716217
0.045364641792095206
9.2148511043888e-15
0.0
Doing team: 6111
Doing team: 5892
Doing team: 5070
Doing team: 231
Finding probabilities
0.7602499389065231
0.0786496035251425
0.0
0.0
Doing team: 7418
Doing team: 5923
Doing team: 5829
Doing team: 3335
Finding probabilities
0.03796348149127882
1.727047068467158e-05
0.0
0.0
Doing team: 2587
Doing team: 7616
Doing team: 7494
Doing team: 4587
Doing team: 441
Doing team: 6645
Doing team: 5908
Doing team: 5261
Finding probabilities
0.02275013194817921
9.865877004244794e-10
0.0
0.0
Doing team: 2882
Doing team: 418
Finding probabilities
0.7181485691746134
0.16796190657586785
0.0
0.0
Doing team: 8370
Doing team: 8274
Doing team: 8210
Doing team: 5427
Finding probabilities
0.17366176642787012
0.0013937377917365712
0.0
0.0
Doing team: 4328
Doing team: 324
Finding probabilities
0.8409854191831092
0.6242274435828845
0.0009908391849000608
3.948974480749712e-11
Doing team: 8098
Doing team: 1255
D